Importing required libraries for data and chi square value

In [ ]:
import pandas as pd
import numpy as np

#for chi square value from table
from scipy.stats import chi2

Importing and processing the training data

In [ ]:
#adding names to attributes
att = ["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","A16","A17","A18","A19","A20","A21","A22","A23","A24","A25","A26","A27","A28","A29","A30","A31","A32","A33","A34","A35","A36","A37","A38","A39","A40","A41","A42","A43","A44","A45","A46","A47","A48","A49","A50","A51","A52","A53","A54","A55","A56","A57","A58","A59","A60","Class"]

#importing training data
t_data = pd.read_csv(r'train.csv',sep=',',index_col=0,header=None)

#splitting the sequence of characters into separate attributes
df = t_data[1].str.split('',n=60,expand=True)

#dropping the index column
df.drop([0],inplace=True,axis=1)

#adding the labels/classes column to the dataframe
df[61]=t_data[2]

#adding attribute names to each column
df.columns=att

To find the major class in the training data

In [ ]:
#finding the major class in initial data
parent_class = np.unique(df['Class'])[np.argmax(np.unique(df['Class'],return_counts=True)[1])]

Calculating the impurity of data

In [ ]:
#function to square a number
def square(x):
    return x*x

#function to calculate impurity given the method and 
#array of count of labels for an attribute and its value
def impurity(method,label_counts):
    imp=0

    #using gini index
    if method=="gini":
        imp=1-(sum(map(square,label_counts))/square(sum(label_counts)))

    #using entropy
    elif method=="entropy":
        for i in range(label_counts.size):
            #trying to avoid zeroes to prevent runtime error of log2(0)
            if label_counts[i]!=0:
                imp=imp+label_counts[i]*np.log2(label_counts[i])
        imp = -(imp)

    #using mis-classification error
    elif method=="mis":
        imp_d=1-((label_counts[0]**2+label_counts[1]**2+label_counts[2]**2)/sum(label_counts)**2)
    else:
        pass
        
    return imp

Calculating the information gain

In [ ]:
#function to calculate information gain given data set and the attribute
def information_gain(current_data,col):

    #to calculate the count of classes in given data
    class_name, class_count = np.unique(current_data['Class'],return_counts=True)

    #a temporary dictionary to store the counts of classes
    dit={}
    for i in range(class_name.size):
        dit[class_name[i]]=class_count[i]

    #if there is no count of any of the classes,make it as zero
    if "EI" not in class_name:
        dit['EI']=0
    if "IE" not in class_name:
        dit['IE']=0
    if "N" not in class_name:
        dit['N']=0
    dit=dict(sorted(dit.items()))

    #this gives the count of classes including zeroes
    class_count=np.array(list(dit.values()))

    #calculating the impurity of given data set using gini index
    imp_d=impurity("gini",class_count) #the impurity method is modifiable here

    #an array to store the impurities for the subsets of the given data
    arr_imp=[]

    #dividing the data set based on the attribute values
    att_value, att_count = np.unique(current_data[col],return_counts=True)

    #for each value of the attribute
    for i in att_value:
        #calculating the count of classes for each value of the attribute
        class_names, class_counts=np.unique(current_data[current_data[col]==i]['Class'],return_counts=True)

        #a temporary dictionary to store the counts of classes
        di={}
        for j in range(class_names.size):
            di[class_names[j]]=class_counts[j]

        #if there is no count of any of the classes,make it as zero
        if "EI" not in class_names:
            di['EI']=0
        if "IE" not in class_names:
            di['IE']=0
        if "N" not in class_names:
            di['N']=0

        di=dict(sorted(di.items()))
        class_counts=np.array(list(di.values()))

        #calculating the impurities and storing in array of impurities
        arr_imp.append(impurity("gini",class_counts))
    #calculating the information gain
    s=0
    for k in range(att_count.size):
        s=s+(att_count[k]*arr_imp[k])
    ig=imp_d-(s/sum(att_count))

    return ig

Calculating the chi square value

In [ ]:
#function to calculate chi square value and compare with the value from table
def chi_square(current_data,col):

    #initial chi square value
    chi_value=0

    #to calculate the count of classes in given data
    class_name, class_count = np.unique(current_data['Class'],return_counts=True)

    #dividing the data set based on the attribute values
    att_value, att_count = np.unique(current_data[col],return_counts=True)

    #for each value of the attribute
    for i in att_value:
        ind=list(att_value).index(i)

        #count of the attribute value
        value_count=att_count[ind]

        #calculating the count of classes for each value of the attribute
        class_names, class_counts=np.unique(current_data[current_data[col]==i]['Class'],return_counts=True)

        #calculating the real and expected counts of a class for a value of the attribute
        for n in range(class_counts.size):
            real=class_counts[n]
            expected=value_count*class_count[n]/sum(class_count)
            chi=square(real-expected)/expected
            chi_value+=chi

    #calculating the degrees of freedom
    dof=(att_value.size-1)*(class_name.size-1)

    #calculating chi square value from table using confidence level and dof
    chi_table=chi2.ppf(0.99, dof) #confidence level can be 0.0, 0.95, 0.99

    #if the calculated value is greater than that of from the table, the split is not random
    if chi_value > chi_table:
        return True
        
    #otherwise the split is random and will not be continued
    else:
        return False

ID3 Algorithm to build decision tree

In [ ]:
#function to build the decision tree for a given data set
def decision_tree(data_set):

    #finding the classes exist in given data set
    class_names=np.unique(data_set['Class'])

    #if all the data belongs to the same class, it returns that class
    if class_names.size==1:
        return class_names[0]

    #if there are no more attributes to choose, it returns the parent node class
    elif len(data_set.columns)==1:
        return parent_class

    else:
        #otherwise, the class for the node is the major class in the given data
        parent_class = np.unique(data_set['Class'])[np.argmax(np.unique(data_set['Class'],return_counts=True)[1])]

        #a temporary dictionary to store the information gain for all attributes
        IG={}

        #iterating through all the columns of given data
        for index,col in zip(range(data_set.columns.size-1),data_set.columns):
            #calculating the information gain using the function
            IG[col]=information_gain(data_set,col)

        #the best attribute with maximum ig is stored in the 'attribute' variable
        attribute=max(IG,key=IG.get)

        #'tree' is the final dictionary that stores the entire tree
        #the attribute is added to the tree
        tree={attribute:{}}

        #performing the chi square test
        if(chi_square(data_set,attribute)):
            #if the test passes, the data is split into sub data based on the attribute values
            values=np.unique(data_set[attribute])

            #for each attribute value
            for value in values:
                #a new data set to store the sub data
                data_set1=data_set[data_set[attribute]==value]

                #the attribute is dropped from the sub data as it is already choosen
                data_set1=data_set1.drop([attribute],axis=1)

                #the sub data is passed to the algorithm again to generate new tree
                tree1 = decision_tree(data_set1)

                #the new tree is added to the main tree
                tree[attribute][value] = tree1
            #returns the final decision tree in the form of dictionary
            return tree
        else:
            #if the test fails, it returns the parent node class
            return parent_class

Function to perform prediction of testing data

In [ ]:
#function to predict the classes for testing data 
#which takes the sequence of attribute values for a sample and the final tree
def prediction(sequence,tree):

    #the first attribute from the tree is stored in 'attribute variable'
    attribute=list(tree.keys())[0]

    #finding the index of this attribute in all the columns of the data set
    index_att=att.index(attribute)

    #checks if the value at the index in given sequence exists in the values of that attribute in the tree
    if sequence[index_att] in list(tree[attribute].keys()):
        #if exists, the subtree for that value of the attribute is stored in 'sub_tree'
        sub_tree = tree[attribute][sequence[index_att]]
        
        #checks if the sub_tree is a tree or a class label
        if isinstance(sub_tree,dict):
            #if it is a tree, the prediction is performed recursively using this sub_tree
            return prediction(sequence,sub_tree)
        elif sub_tree is not None:
            #if it is not a tree, it returns the sub_tree which could be a class label
            return sub_tree
        else:
            #if all the conditions fail, it returns the parent node class
            return parent_class
    else:
        #if the value does not exist in the tree, it returns the parent node class
        return parent_class

Training the classifier

In [ ]:
#passing the main data set to the decision tree function
final_tree = decision_tree(df)

Testing the classifier

In [ ]:
#importing the testing data
test_data_set = pd.read_csv(r'test.csv', header=None)

#resetting the index
test_data_set = test_data_set.reset_index()

#passing each row to the prediction function, for all the rows in the data set
final_prediction = [[row[0], prediction(row[1], final_tree)] for index,row in test_data_set.iterrows()]

#to export the final prediction to a csv file with header names
pd.DataFrame(final_prediction).to_csv('prediction.csv', header=["id","class"], index=False)